# Prática Independente - Regressão IV.

## Dados abertos do Airbnb no Rio de Janeiro:

#### Desde 2008, os hóspedes e anfitriões têm usado o Airbnb para expandir as possibilidades de viagem e apresentar uma forma mais única e personalizada de experimentar o mundo. Este conjunto de dados descreve a atividade de listagem e as métricas em Rio de Janeiro, RJ para 2020.

### Conteúdo:

#### O arquivo `'AB_RJ_2020.csv'` inclui todas as informações necessárias para descobrir mais sobre hosts, disponibilidade geográfica, métricas necessárias para fazer previsões e tirar conclusões.

#### Este conjunto de dados [públicos](http://insideairbnb.com/) faz parte do [Airbnb](https://www.airbnb.com.br/).

<img src="RegMetroRJ.png" width="1532" height="1037" align="center"/>

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# Tamanho padrões das figuras e a fonte de seus textos neste notebook
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 14

In [1]:
def EDA (df):
    eda_df = {}
    eda_df['Amount_NaN'] = df.isnull().sum()
    eda_df['%_NaN'] = df.isnull().mean().round(2)
    eda_df['DType'] = df.dtypes
    eda_df['Amount_Data'] = df.count()
    
    # Outro ponto para ser verificado, porque para criar a coluna com a quantidade de valores unicos por coluna
    # Não utilizei a função df.unique() 
    colunas = sorted(df.columns.tolist(), key=str.lower, reverse=False)
    eda_df['Amount_Unique'] = filtered_result = list(map(lambda x: len(df[x].unique().tolist()), colunas))
    
    eda_df['Mean'] = np.round(df.mean(), 2)
    eda_df['Median'] = np.round(df.median(), 2)
    
    eda_df['Max'] = df.max()
    eda_df['Min'] = df.min()
    eda_df['STD'] = np.round(df.std(), 2)
    
    return pd.DataFrame(eda_df)

#### Exercício 1 - Realize uma análise explanatória nos dados abertos do AirBNB para a cidade do Rio de Janeiro.

In [4]:
dados = pd.read_csv('AB_RJ_2020.csv', parse_dates= True, index_col= 'id')
#dados.head()

In [5]:
print(f'Instâncias: {dados.shape[0]}')
print(f'Variáveis: {dados.shape[1]}')

Instâncias: 25784
Variáveis: 14


In [6]:
EDA(dados)

,Amount_NaN,%_NaN,DType,Amount_Data,Amount_Unique,Mean,Median,Max,Min,STD
availability_365,0,0.00,int64,25784,366,215.30,247.00,365,0,1.408900e+02
calculated_host_listings_count,0,0.00,int64,25784,48,6.58,1.00,200,1,2.000000e+01
host_id,0,0.00,int64,25784,16946,96308131.49,58502919.50,381289871,3607,1.041834e+08
host_name,8,0.00,object,25776,5076,NaN,NaN,NaN,NaN,NaN
last_review,9932,0.39,object,15852,1447,NaN,NaN,NaN,NaN,NaN
latitude,0,0.00,float64,25784,9782,-22.97,-22.97,-22.7498,-23.0729,3.000000e-02
longitude,0,0.00,float64,25784,12093,-43.25,-43.20,-43.1049,-43.7048,1.000000e-01
minimum_nights,0,0.00,int64,25784,67,4.77,2.00,1000,1,1.904000e+01
name,29,0.00,object,25755,25086,NaN,NaN,NaN,NaN,NaN
neighbourhood,11675,0.45,object,14109,270,NaN,NaN,NaN,NaN,NaN


1) Temos 25.784 instâncias no dataset e 14 variáveis a priori;

2) Temos muitos dados nulos nas variáveis last_review, neighbourhood e reviews_per_month

3) Precisamos tratar os dados da coluna price para tranformá-los em tipo Float

4) Temos uma media de 215 em que os hosts estão disponíveis, por ano, com uma mediana de 247 e moda de 365. A maioria está disponivel o ano todo, porém possuímos outliers com o número de 0 disponíveis por ano que podem estar puxando a media pra baixo.

5) Olhando mediana e media podemos indicar que a maioria dos dados de minímo de noite (para a paga do valor indicado) está entre 2 e 5).

In [7]:
dados['host_name'].value_counts()

Daniel               318
Maria                270
Marcelo              243
Ricardo              212
Carlos               196
                    ... 
Marcelo Caetano        1
Rosilea                1
Isaia                  1
Iran                   1
Marcus Vinicius A      1
Name: host_name, Length: 5075, dtype: int64

Ao contar o número de hosts por pessoas vemos muitas que se repetem. Isso pode se dar porque só o primeiro nome está contido e assim são pessoas diferentes com nomes iguais, ou pode haver pessoas com maior número de locais disponiveis na plataforma Airbnd e outras que colocam a vaga de forma mais eventual.

#### Exercício 2 -Faça a limpeza dos dados.

In [8]:
#Modificando coluna de preço de object para float
dados['price']= dados['price'].str.replace('$', '', regex= True)
dados['price']= dados['price'].str.replace(',', '', regex= True)
dados['price']= dados['price'].str.replace('.', '', regex= True)
dados['price'] = pd.to_numeric(dados['price'])

dados['price'].dtype

dtype('int64')

Imputar os valores faltantes em neighbourhood usando os vizinhos mais próximos em Longitude e Latitude.

#### Exercício 3 - Analise os dados e realize a anonimização dos mesmos. 

#### Exercício 4 - Crie um modelo de regressão para os preços dos apartamentos.